In [3]:
import os
import csv
# import umap
import json
import librosa
import numpy as np
import matplotlib.pyplot as plt
# from sklearn import preprocessing
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import MinMaxScaler
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen

In [11]:
import file_load
video_path = "./dataset"
name = ".mp3"

file_list, file_name, folder_list, folder_name = file_load.file_path(video_path, name)

for mp3, name in zip(file_list, file_name):
    save_path = mp3.replace('.mp3','')
    print(save_path)
    wave, sr = librosa.load(mp3,sr = 16000)
    audio_size = np.size(wave)
    print(wave.shape)
    split_size = 480000
    count = 0
    for split_point in range(0, audio_size, split_size):
        if count<1:
            split_audio = wave[split_point : split_point + split_size]
            np.save(save_path+'_{}'.format(count), split_audio)
            print("wave split {} OK".format(split_audio.shape))
        count += 1

./dataset/no8/Old School Turntables
(480654,)
wave split (480000,) OK
./dataset/no8/Say what’s on your mind
(480654,)
wave split (480000,) OK
./dataset/no8/NO THANXXX
(480654,)
wave split (480000,) OK
./dataset/no3/gogobebe
(480654,)
wave split (480000,) OK
./dataset/no3/Egotistic
(480654,)
wave split (480000,) OK
./dataset/no3/Angel
(480654,)
wave split (480000,) OK
./dataset/no4/Hollywood
(480654,)
wave split (480000,) OK
./dataset/no4/SF
(480654,)
wave split (480000,) OK
./dataset/no4/Lie
(480654,)
wave split (480000,) OK
./dataset/no2/Decalcomanie
(480654,)
wave split (480000,) OK
./dataset/no2/ILoveToo
(480654,)
wave split (480000,) OK
./dataset/no2/RainySeason
(480654,)
wave split (480000,) OK
./dataset/no10/44f4a79e49f7ffcec038573e94cdab5d1b98cda6
(480654,)
wave split (480000,) OK
./dataset/no10/6831b5cca17f371165e455369c736d0166336f36
(480654,)
wave split (480000,) OK
./dataset/no10/4224ae32a34215ee7e6e5d144dbe7f418a47e7b0
(480654,)
wave split (480000,) OK
./dataset/no7/wf
(480

In [16]:
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen
import file_load

model_path = './wavenet-ckpt/wavenet-ckpt/model.ckpt-200000' #模型位置
npy_path = './dataset'#dataset位置
name = '.npy' #欲讀取的檔名

def wavenet_encode(file_path):
    audio = np.load(file_path)
    encoding = fastgen.encode(audio, model_path, len(audio))
    print(encoding.reshape((-1, 16)).shape)
    return encoding.reshape((-1, 16))

file_list, file_name, folder_list, folder_name = file_load.file_path(npy_path, name)

for npy, name in zip(file_list, file_name):
    print(npy)
    wavenet_data = wavenet_encode(npy)
    stddev_wavenet = np.std(wavenet_data, axis=0)
    mean_wavenet = np.mean(wavenet_data, axis=0)
    
    average_difference_wavenet = np.zeros((16,))
    for i in range(0, len(wavenet_data) - 2, 2):
        temp = wavenet_data[i] - wavenet_data[i+1]
        average_difference_wavenet += temp
    average_difference_wavenet /= (len(wavenet_data) // 2)   
    average_difference_wavenet = np.array(average_difference_wavenet)
    
    concat_features_wavenet = np.hstack((stddev_wavenet, mean_wavenet))
    concat_features_wavenet = np.hstack((concat_features_wavenet, average_difference_wavenet))
    
    save_path = npy.replace('.npy', "") 
    print('save',concat_features_wavenet.shape)
    np.save(save_path+'_OK',concat_features_wavenet) #存特徵向量
    

./dataset/no8/Old School Turntables_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no8/NO THANXXX_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no8/Say what’s on your mind_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no3/Angel_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no3/gogobebe_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no3/Egotistic_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/no4/Lie_0.npy
INFO:tensorflow:Restoring parameters from ./wavenet-ckpt/wavenet-ckpt/model.ckpt-200000
(937, 16)
save (48,)
./dataset/n

In [52]:
import file_load
import numpy
npy_path = './dataset/'#dataset位置
name = 'OK.npy' #欲讀取的檔名

file_list, file_name, folder_list, folder_name = file_load.file_path(npy_path, name)

for c in range(len(folder_name)):
    fi_list, fi_name, fo_list, fo_name = file_load.file_path(npy_path+'no'+str(c+1), name)
    npy_list = []
    print('no'+str(c+1))
    
    for npy, n in zip(fi_list, fi_name):
        vec = np.load(npy)
        npy_list.append(vec)
        print(n.replace('_0_OK', ''))
        
    if len(npy_list)==3: 
        dist = np.linalg.norm(npy_list[0]-npy_list[1])
        print(dist,'    0對1')
        dist = np.linalg.norm(npy_list[0]-npy_list[2])
        print(dist,'    0對2')
        dist = np.linalg.norm(npy_list[1]-npy_list[2])
        print(dist,'    1對2')
    
    elif len(npy_list)==2:
        dist = np.linalg.norm(npy_list[0]-npy_list[1])
        print(dist)
    print("================")

no1
YOUTH
Bloom
MYMYMY
2.830699914456322     0對1
2.27186095339642     0對2
0.8675373922773715     1對2
no2
RainySeason
ILoveToo
Decalcomanie
1.3239942531157127     0對1
1.8122381616397454     0對2
2.65248001066036     1對2
no3
gogobebe
Egotistic
Angel
0.48026388253221775     0對1
1.788494752006492     0對2
1.5435984406194327     1對2
no4
SF
Lie
Hollywood
1.0652123327911913     0對1
1.041392130835179     0對2
0.6575773697672002     1對2
no5
EpikHigh-Decalcomanie
Colde-Lovestruck
EpikHigh-NOTHANXXX
1.7957430801751673     0對1
0.7965891389392296     0對2
2.0336525322391497     1對2
no6
Wolves
SidetoSide
petedavidson
1.6356742053229116     0對1
2.803708321277067     0對2
1.2793955245668986     1對2
no7
wf
ja
0.22411981138740128
no8
Old School Turntables
Say what’s on your mind
NO THANXXX
2.229191839178503     0對1
2.7560940029973193     0對2
1.2587211266927758     1對2
no9
Break up with your girlfriend
Conan Gray - Grow
Roy Bliar - Grow Up
1.4307349515972636     0對1
1.0103889632938854     0對2
1.86290584360638